# SARSA & Q - Learning

# 연습데이터 : Frozen Lake
![좋은거](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSqnklD5CDrIqDgS28YCAcmQkfBhJVvYirrRbZjmZeta5Dfx4_e&usqp=CAU)

**학습목표**

1. SARSA의 업데이트 맛 본다.
2. Q-learning의 업데이트를 맛 본다.
3. 일단 코드 짤 수 있다.

# 필요 라이브러리 불러오기

1. 이런 연습에서는 딱히 비디오 영상이 필요하지 않다.

In [1]:
!pip install gym

In [2]:
import numpy as np
import gym

# 사용할 환경 불러오기

In [3]:
# gym.envs.registration.register(
#     id="FrozenLake-v3", entry_point='gym.envs.toy_text:FrozenLakeEnv',
#     kwargs={'map_name': '4x4', 'is_slippery': False}
# )

gym.envs.registration.register(
    id="FrozenLake-v8", entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name': '8x8', 'is_slippery': True}
)

In [4]:
# env = gym.make('FrozenLake-v3')
env = gym.make('FrozenLake-v8')
print('observation space:', env.observation_space)
print('action space:', env.action_space)

observation space: Discrete(64)
action space: Discrete(4)


# SARSA와, Q-learning 준비 해보기

**핵심 코드**
1. SARSA : S,A,R,S,A 가 끝나면 Q업데이트!
```
Q[s,a] = Q[s,a] + alpha * ((r + gama* Q[s1,a1]) - Q[s,a])
```
2. Q-Learning : S,A,R,S 후 max A, 그다음 Q업데이트!
```
Q[s,a] = Q[s,a] + alpha*(r + gama*np.max(Q[s1,:]) - Q[s,a])
```


# II. Q-Learning


### 1. Q-Table을 준비하자.

* Q = np.zeros([state의 수, action의 수])
* 활용 예시
    * Q[2, 3]에는 10이 담겨있음.
    * Q[2, 1]에는 5가 담겨있음.
    * 2번 state에선 3번 액션이 1번 액션보다 2배 가치 있음!

* env.action_space.n, env.observation_space.n 이용

In [ ]:
env.observation_space, env.action_space

In [6]:
## Your Code Here
Q = np.zeros(shape=(env.observation_space.n, env.action_space.n,))
Q.shape


(64, 4)

In [7]:
print(Q)
# for rows in Q.reshape([4,4,4]):
#     print(*rows)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### 2. parameter들

In [8]:
alpha = 0.2
gamma = 0.999 # 할인율!
n_episod = 20000
epsilon = 0.15

### 3. q-learning

In [9]:
Q = np.zeros([env.observation_space.n,
              env.action_space.n])

rewards_ql = []

for i in range(n_episod) : 
    s0 = env.reset()
    done = False

    while True :
        # e-greedy 하게 행동을 선택하여 a0에 선언하자.
        if np.random.uniform() < epsilon :
          a0 = env.action_space.sample()
        else :
          greedy_actions = np.argwhere(Q[s0, :] == np.amax(Q[s0, :])).reshape(-1)
          a0 = np.random.choice(greedy_actions)

        # a0 를 이용하여 
        # s1, r1, done, _ 를 새로 선언하다.
        s1, r1, done, _ = env.step(a0)

        # ( s0, a0, r1, s1 ) 로 부터! 차이점 주의!
        # 업데이트 코드를 작성하자.
        Q[s0, a0] = Q[s0, a0] + alpha * (r1 + gamma*np.max(Q[s1,:]) - Q[s0, a0])

        if done == True: # 종료되었으면
            rewards_ql.append(r1)
            env.close() # 환경 닫고.
            break # 멈춰야지

        s0 = s1 # 다음 루프에선 이것이 직전 state

    if (i+1) % 4000 == 0 :
        print('===========  에피소드 : {}  ============'.format(i+1))
        # for rows in Q.reshape([-1,4,4]):
        #     print(*rows)
        

===========  에피소드 : 4000  ============
===========  에피소드 : 8000  ============
===========  에피소드 : 12000  ============
===========  에피소드 : 16000  ============
===========  에피소드 : 20000  ============


### 학습 완료된 것 구경

위 에서, 학습을 1 ~ 2회만 시켜보고 아래를 해봐도 좋다.

In [10]:
s0 = env.reset()
done = False
a0 = env.action_space.sample() # 처음엔 랜덤무브

while True :
    env.render()
    s1, r1, done, _ = env.step(a0) # 일단 움직인다.

    greedy_actions = np.argwhere(Q[s1, :] == np.amax(Q[s1, :])).reshape(-1)
    a1 = np.random.choice(greedy_actions) # 무슨 뜻?

    if done == True: # 종료되었으면
        env.render()
        env.close() # 환경 닫고.
        break # 멈춰야지

    s0 = s1 # 다음 루프에선 이것이 직전 state
    a0 = a1 # 다음 루프에선 이것이 직전 action


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFF

### Q-learning 학습 코드를 보고 진행 순서를 정리하자.

1. env.reset()을 통해 state 확인
2. epsilon 15% 확률로 random action(exploration)이나 최적 action(exploitation) 선택
  * 최적 action을 경우 
  * (sarsa)
    * 매 action 마다 ( s0, a0, r1, s1, a1 ) 로 부터 아래와 같이 Q 테이블 업데이트
    * Q[s0, a0] = Q[s0, a0] + alpha * (r1 + gamma*Q[s1, a1] - Q[s0, a0])
  * (Q-learning)
    * 매 action 마다 ( s0, a0, r1, s1 ) 로 부터 아래와 같이 Q 테이블 업데이트
    * Q[s0, a0] = Q[s0, a0] + alpha * (r1 + gamma*np.max(Q[s1,:]) - Q[s0, a0])
3. env.step을 통해 다음 state, 다음 reward를 확인
4. done == True 이면 종료